In [1]:
# !mkdir ../data
# !wget -O ../data/camelyonpatch_level_2_split_test_x.h5.gz https://zenodo.org/record/2546921/files/camelyonpatch_level_2_split_test_x.h5.gz?download=1
# !wget -O ../data/camelyonpatch_level_2_split_test_y.h5.gz https://zenodo.org/record/2546921/files/camelyonpatch_level_2_split_test_y.h5.gz?download=1    
# !wget -O ../data/camelyonpatch_level_2_split_valid_x.h5.gz https://zenodo.org/record/2546921/files/camelyonpatch_level_2_split_valid_x.h5.gz?download=1
# !wget -O ../data/camelyonpatch_level_2_split_valid_y.h5.gz https://zenodo.org/record/2546921/files/camelyonpatch_level_2_split_valid_y.h5.gz?download=1 

# import gzip
# import shutil

# with gzip.open('../data/camelyonpatch_level_2_split_test_y.h5.gz', 'rb') as f_in:
#     with open('../data/camelyonpatch_level_2_split_test_y.h5', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)
        
# with gzip.open('../data/camelyonpatch_level_2_split_valid_x.h5.gz', 'rb') as f_in:
#     with open('../data/camelyonpatch_level_2_split_valid_x.h5', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

# with gzip.open('../data/camelyonpatch_level_2_split_valid_y.h5.gz', 'rb') as f_in:
#     with open('../data/camelyonpatch_level_2_split_valid_y.h5', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

# with gzip.open('../data/camelyonpatch_level_2_split_test_x.h5.gz', 'rb') as f_in:
#     with open('../data/camelyonpatch_level_2_split_test_x.h5', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [ ]:
import tensorflow as tf
# !pip install tensorflow-gpu==2.3.4 
# !pip install tensorflow==2.3.4
print(tf.version.VERSION)
tf.config.list_physical_devices('GPU')


In [ ]:
import matplotlib.pyplot as plt
import random
import numpy as np
import numpy
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Add, Dense, Activation, BatchNormalization, Conv2D
from keras.layers import Flatten
from keras.layers.convolutional import MaxPooling2D
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from keras.utils import np_utils
from keras.initializers import glorot_uniform
%matplotlib inline

In [ ]:
model_type = 'vgg16'

In [ ]:
model_home_dir = os.path.join(os.getcwd(), 'CNN_TASK_ODED',"model_type")
print(model_home_dir)
if not os.path.isdir(model_home_dir):
    print('Inside mkdir')
    os.makedirs(model_home_dir)

In [ ]:
#Defining Variables

#Data set information
DATASET = 'tfds.image_classification.Malaria'
num_classes = 2
input_shape=(96,96,3)
epochs = 200
lrate = 0.01
decay = lrate/epochs
batch_size = 32
seed = 7
np.random.seed(seed)

validate_dir = os.path.join("data","test")
train_dir = os.path.join("data","train")

# Prepare model model saving directory.
save_dir = os.path.join(model_home_dir, 'saved_models')

model_name = DATASET+'_'+'%s_model.{epoch:03d}.h5' % model_type

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)
print("Model Type: ",model_type)
print("Model File: ",filepath)

In [ ]:
# print(tf.version.VERSION)

# from tensorflow.python.keras.utils.io_utils import HDF5Matrix
# datapath = '../data/'

# x_valid = HDF5Matrix('../data/camelyonpatch_level_2_split_valid_x.h5','x')
# y_valid = HDF5Matrix('../data/camelyonpatch_level_2_split_valid_y.h5','y')
# y_test = HDF5Matrix('../data/camelyonpatch_level_2_split_test_y.h5','y')
# x_test = HDF5Matrix('../data/camelyonpatch_level_2_split_test_x.h5','x')

# print(y_valid.shape, y_test.shape)
# print(x_valid.shape, x_test.shape)

In [ ]:
# import numpy as np
# from sklearn.model_selection import train_test_split


# X = np.concatenate((x_test, x_valid))
# Y = np.concatenate((y_test, y_valid)).reshape(-1,1)

# # X, y = shuffle(X, Y, random_state=0)

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15)

# print(y_train.shape, y_test.shape)
# print(X_train.shape, X_test.shape)

In [ ]:
# from numpy import save
# save('../data/lymph/y_train.npy', y_train)
# save('../data/lymph/y_test.npy', y_test)
# save('../data/lymph/X_train.npy', X_train)
# save('../data/lymph/X_test.npy', X_test)

In [ ]:
from numpy import load

y_train = load('../data/lymph/y_train.npy')
y_test =  load('../data/lymph/y_test.npy')
X_train = load('../data/lymph/X_train.npy')
X_test =  load('../data/lymph/X_test.npy')

In [ ]:
print(y_train.shape, y_test.shape)
print(X_train.shape, X_test.shape)
"""(55705, 1) (9831, 1)
(55705, 96, 96, 3) (9831, 96, 96, 3)"""

In [ ]:
class_names = ['parasitized', 'uninfected']

print("------------------concatenate train-------------------")
plt.figure(figsize=(10,10))
for i in range(25):
    j = random.randint(0,55705)
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[j])
    plt.xlabel(class_names[y_train[j][0].astype('int')])
plt.show()

print("------------------concatenate test-------------------")
plt.figure(figsize=(10,10))
for i in range(25):
    j = random.randint(0,9831) 
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_test[j])
    plt.xlabel(class_names[y_test[j][0].astype('int')])
plt.show()

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')


X_train_mean = np.mean(X_train)
X_train -= X_train_mean
X_test_mean = np.mean(X_test)
X_test -= X_test_mean

y_train = np_utils.to_categorical(y_train,)
y_test = np_utils.to_categorical(y_test,)
y_train = np.delete(y_train,1,1)
y_test = np.delete(y_test,1,1)
print(y_train.shape, y_test.shape)




In [ ]:
import seaborn as sns
import pandas as pd


plt.title('Distribution')
sns.countplot(y = 0, data=pd.DataFrame(y_test))
plt.show()

In [ ]:
from tensorflow.keras.layers import Dropout
def vgg16(input_shape=(96,96,3), num_classes=2):
  model = Sequential()
  model.add(Conv2D(input_shape=input_shape,filters=64,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))


  model.add(Dropout(0.8))

  model.add(Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  # model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  # model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Flatten())
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=1, activation="sigmoid"))

  return model

In [ ]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


In [ ]:
# Prepare callbacks for model saving and for learning rate adjustment.

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler,early_stopping]


In [ ]:
 model = vgg16(input_shape,num_classes)

# Compile model

# optimizer = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss= 'binary_crossentropy' , optimizer=optimizer, metrics=['accuracy'])
print("Model Summary of ",model_type)
print(model.summary())



In [ ]:
# Score trained model.
history = model.fit(X_train, y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_data=(X_test, y_test),
      shuffle=True,
      callbacks=callbacks)
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

y_pred = model.predict(X_test).ravel()

nn_fpr_keras, nn_tpr_keras, nn_thresholds_keras = roc_curve(y_test, y_pred)
auc_keras = auc(nn_fpr_keras, nn_tpr_keras)
plt.plot(nn_fpr_keras, nn_tpr_keras, marker='.', label='Neural Network (auc = %0.3f)' % auc_keras)
plt.title('ROC Curve')
plt.ylabel('TPR')
plt.xlabel('FPR')
plt.show()

print(auc_keras)

In [ ]:
from matplotlib import pyplot
predict_x=model.predict(X_test) 

parasitized_index = np.where(y_test.reshape(-1) == 0.)
parasitized = predict_x[parasitized_index]

uninfected_index = np.where(y_test.reshape(-1) == 1.)
uninfected = predict_x[uninfected_index]



bins = np.linspace(0, 1, 100)

pyplot.hist(parasitized, bins, alpha=0.5, label='parasitized')
pyplot.hist(uninfected, bins, alpha=0.5, label='uninfected')
pyplot.legend(loc='upper right')
pyplot.show()

In [ ]:
predict_test=model.predict(X_test)
predict_train=model.predict(X_train) 

parasitized_index = np.where(y_test.reshape(-1) == 0.)
num_parasitized = len(predict_test[parasitized_index]) + len(predict_train[parasitized_index])

uninfected_index = np.where(y_test.reshape(-1) == 1.)
num_uninfected = len(predict_test[uninfected_index]) + len(predict_train[uninfected_index])

print(f"number of parasitized:{num_parasitized}")
print(f"number of uninfected:{num_uninfected}")


In [ ]:
predict_test=model.predict(X_test)
predict_train=model.predict(X_train)
print(len(predict_test))
print(len(predict_train))


parasitized_index = np.where(y_test.reshape(-1) == 0.)
print(len(predict_test[parasitized_index]))
print(len(predict_train[parasitized_index]))

uninfected_index = np.where(y_test.reshape(-1) == 1.)
print(len(predict_test[uninfected_index]))
print(len(predict_train[uninfected_index]))


In [ ]:
print("number of uninfected:")
predict_train=model.predict(X_train) 

In [ ]:
len(parasitized)

In [ ]:
len(uninfected)

In [ ]:
# for X,Y in zip(predict_x,y_test):
#     print("Model Score:", X, "actual score:", Y) 

In [ ]:
len(predict_x)

In [ ]:
predict_y=model.predict(X_train) 
len(predict_y)